http://brandonrose.org/clustering#Tf-idf-and-document-similarity

In [12]:
import pandas as pd
import numpy as np
import collections
import json

In [13]:
lda_result = pd.read_csv('./lda.tsv', sep='\t')
km_result = pd.read_csv('./km.tsv', sep='\t')

In [16]:
# km_result = pd.DataFrame([0, 6, 0, 6, 6, 6, 6, 0, 0, 6, 0, 0, 0, 6, 0, 6, 6, 6, 6, 0, 0, 0,
#        0, 0, 2, 6, 6, 0, 6, 6, 2, 0, 0, 0, 0, 6, 4, 2, 6, 7, 0, 6, 6, 0,
#        5, 0, 6, 6, 6, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        2, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 6, 2, 2, 2, 2,
#        6, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2,
#        2, 2, 6, 2, 6, 2, 2, 2, 2, 4, 2, 2, 6, 2, 2, 2, 6, 2, 2, 2, 2, 2,
#        4, 2, 4, 4, 4, 4, 0, 7, 7, 7, 4, 4, 0, 2, 4, 0, 7, 7, 4, 7, 0, 4,
#        4, 2, 4, 7, 0, 7, 0, 0, 0, 4, 4, 2, 7, 0, 4, 4, 4, 0, 0, 4, 4, 0,
#        4, 0, 4, 4, 4, 4, 2, 4, 5, 6, 6, 6, 5, 5, 6, 5, 5, 5, 5, 5, 6, 5,
#        6, 5, 7, 5, 6, 7, 5, 0, 0, 6, 6, 6, 6, 5, 5, 5, 0, 6, 6, 6, 6, 2,
#        7, 7, 5, 5, 7, 5, 6, 2, 6, 6, 7, 0, 5, 7, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        4, 0, 0, 0, 4, 4, 7, 7, 0, 7, 7, 2, 7, 7, 7, 2, 7, 7, 2, 7, 7, 2,
#        7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7, 2, 7, 7, 7, 6, 7, 2, 7, 7, 7, 7,
#        7, 7, 7, 7], columns=['topic']).reset_index()
# km_result.columns = ['id', 'topic']

In [14]:
def intersect(a, b):
    return len(list(set(a) & set(b)))

In [15]:
lda_docs_by_topic = lda_result.groupby('topic').agg({'id': 'unique'})
km_docs_by_topic = km_result.groupby('topic').agg({'id': 'unique'})

In [16]:
lda_docs_by_topic.head()

,id
topic,
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,"[50, 52, 53, 54, 55, 60, 62, 64, 65, 68, 70, 7..."
2,"[51, 56, 57, 58, 59, 61, 63, 67, 73, 74, 84, 2..."
3,"[69, 163, 251, 256, 280, 282, 283, 289, 296, 2..."
4,"[195, 201, 202, 207, 209, 210, 211, 214, 216, ..."


In [17]:
km_docs_by_topic.head()

,id
topic,
0,"[25, 50, 51, 53, 54, 60, 63, 65, 66, 69, 75, 7..."
1,"[101, 102, 103, 104, 105, 106, 107, 108, 109, ..."
2,"[350, 351, 352, 353, 354, 355, 356, 357, 358, ..."
3,"[1, 3, 8, 10, 14, 31, 35, 36, 154, 155, 161, 2..."
4,"[83, 87, 91, 96, 195, 200, 201, 202, 203, 204,..."


In [18]:
lda_km_topic_map = {} # key: km, value: lda

In [19]:
for i in range(8):
    lda_km_topic_map[i] = np.argmax([intersect(km_docs_by_topic.iloc[i].values[0], lda_docs_by_topic.iloc[j].values[0]) for j in range(8)])
    print('KM topic' + str(i+1))
    print([intersect(km_docs_by_topic.iloc[i].values[0], lda_docs_by_topic.iloc[j].values[0]) for j in range(8)])
    print('LAD topic' + str(np.argmax([intersect(km_docs_by_topic.iloc[i].values[0], lda_docs_by_topic.iloc[j].values[0]) for j in range(8)]) + 1))

KM topic1
[4, 18, 4, 16, 9, 6, 17, 9]
LAD topic2
KM topic2
[0, 0, 0, 0, 0, 0, 46, 1]
LAD topic7
KM topic3
[0, 0, 0, 32, 0, 0, 0, 0]
LAD topic4
KM topic4
[16, 1, 0, 0, 0, 41, 1, 3]
LAD topic6
KM topic5
[1, 4, 19, 0, 30, 0, 0, 1]
LAD topic5
KM topic6
[0, 9, 8, 0, 0, 0, 0, 0]
LAD topic2
KM topic7
[3, 1, 1, 4, 0, 6, 1, 39]
LAD topic8
KM topic8
[41, 0, 1, 4, 0, 0, 0, 3]
LAD topic1


In [21]:
for k in lda_km_topic_map:
    print('LDA topic %d = K-Means topic %d' % (lda_km_topic_map[k], k))

LDA topic 1 = K-Means topic 0
LDA topic 6 = K-Means topic 1
LDA topic 3 = K-Means topic 2
LDA topic 5 = K-Means topic 3
LDA topic 4 = K-Means topic 4
LDA topic 1 = K-Means topic 5
LDA topic 7 = K-Means topic 6
LDA topic 0 = K-Means topic 7


In [24]:
merged_result = pd.merge(lda_result[['id', 'document', 'topic']], km_result[['id', 'topic']], on='id', suffixes=('_lda', '_km'))
merged_result.topic_km = merged_result.topic_km.apply(lambda x: lda_km_topic_map[x])

In [25]:
merged_result.head()

,id,document,topic_lda,topic_km
0,0,Hazard Ratio (and 95% Confidence Intervals) ...,0,0
1,1,Left Ventricular and Right Ventricular Ejectio...,0,5
2,2,Partial Regression Coefficients (All Subjects...,0,0
3,3,Left Ventricular (LVEF) and Right Ventricula...,0,5
4,4,Predictors of Mortality by Multivariable Analy...,0,0


In [26]:
json_data = collections.OrderedDict()
json_data['rows'] = merged_result[['document', 'topic_lda', 'topic_km']].to_dict(orient='records')
jsonp = json.loads(json.dumps(json_data, ensure_ascii=False, indent='\t').replace('`', ''))

In [27]:
f = open('./document_table.json', 'w')
f.write(json.dumps(json_data, ensure_ascii=False, indent='\t').replace('`', ''))
f.close()